In [2]:
!pip install --disable-pip-version-check -q awswrangler==2.14.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-expr 1.0.11 requires pandas>=2, but you have pandas 1.3.5 which is incompatible.
dask-expr 1.0.11 requires pyarrow>=7.0.0, but you have pyarrow 6.0.1 which is incompatible.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data-insights 0.3.3 which is incompatible.
sphinx 7.2.6 requires docutils<0.21,>=0.18.1, but you have docutils 0.16 which is incompatible.


In [3]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0

In [4]:
import boto3
import sagemaker
import pandas as pd
from pyathena import connect

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
s3 = boto3.resource('s3')

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [5]:
s3_private_path_csv = "s3://{}/dataset/csv".format(bucket)
print(s3_private_path_csv)

s3://sagemaker-us-east-1-652903355321/dataset/csv


In [6]:
import os
# current working directory
current_directory = os.getcwd()
# elative path to the data file within the current directory
file_path = os.path.join(current_directory, 'dataset.csv')
print("Path of the data file:", file_path)

Path of the data file: /root/aai-540-labs/MLOpsFinalProject/dataset.csv


In [7]:
s3.Object(bucket, 'dataset/csv/dataset.csv').put(Body=open('/root/aai-540-labs/MLOpsFinalProject/dataset.csv', 'rb'))

{'ResponseMetadata': {'RequestId': 'CBF1JWTMRQ1Z0X9P',
  'HostId': 'xISpLSZCdKf7K9beDwS4acgTpSKe12Hbd9taqsAGX3BtTtIgOKOsvbexABW9/EFSoiHpDB9oJ+4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'xISpLSZCdKf7K9beDwS4acgTpSKe12Hbd9taqsAGX3BtTtIgOKOsvbexABW9/EFSoiHpDB9oJ+4=',
   'x-amz-request-id': 'CBF1JWTMRQ1Z0X9P',
   'date': 'Tue, 14 May 2024 03:07:48 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"e50a70711e823b75c8ebe231468f5faf"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"e50a70711e823b75c8ebe231468f5faf"',
 'ServerSideEncryption': 'AES256'}

In [8]:
!aws s3 ls $s3_private_path_csv/

2024-05-14 03:07:48   20118244 dataset.csv


In [9]:
!aws s3 ls s3://sagemaker-us-east-1-652903355321

                           PRE FinalProjectDataset/
                           PRE amazon-reviews-pds/
                           PRE athena/
                           PRE dataset/
2024-05-11 15:42:49   20118244 dataset.csv


In [10]:
import awswrangler as wr

In [11]:
df_csv_results = wr.s3.read_csv(s3_private_path_csv, dataset=True)
df_csv_results.head(5)

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [12]:
# column album_name and track_name has , in their values which might cause issues hence replacing it with space
columns_to_clean = ['album_name', 'track_name']
df_csv_results[columns_to_clean] = df_csv_results[columns_to_clean].replace(',', '', regex=True)

In [13]:
#datafrane after removing , values
df_csv_results.head(5)

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [17]:
# 1. List artist, track_name, and popularity for songs that have a popularity greater than or equal to 99

In [14]:

# Filter the DataFrame to select rows with popularity >= 99
popularity_df = df_csv_results[df_csv_results['popularity'] >= 99]

# Select artist, track_name, and popularity
popular_result_df = popularity_df[['artists', 'track_name', 'popularity']]


# Print popularity >= 99
print(popular_result_df)

                    artists                            track_name  popularity
20001  Sam Smith;Kim Petras             Unholy (feat. Kim Petras)         100
51664      Bizarrap;Quevedo  Quevedo: Bzrp Music Sessions Vol. 52          99
81051  Sam Smith;Kim Petras             Unholy (feat. Kim Petras)         100


In [19]:
# 2. List artists with an average popularity of 92

In [15]:
#Calculate average popularity by artists
average_popularity = df_csv_results.groupby('artists')['popularity'].mean()
print(average_popularity.head(5))

artists
!nvite                                                23.000000
"Puppy Dog Pals" Cast                                 57.500000
"Weird Al" Yankovic                                   26.333333
#Kids;Nursery Rhymes;Nursery Rhymes and Kids Songs    34.000000
$affie                                                39.000000
Name: popularity, dtype: float64


In [16]:
#rceate datafrane with average popularity of 92
average_popularity_92=average_popularity[average_popularity == 92]
print(average_popularity_92)

artists
Harry Styles         92.0
Rema;Selena Gomez    92.0
Name: popularity, dtype: float64


In [17]:
# 3.List the Top 10 most energetic genres

In [18]:
#calculate average energy for track_genre
average_energy_genre=df_csv_results.groupby('track_genre')['energy'].mean()
print(average_energy_genre.head(5))

track_genre
acoustic       0.435368
afrobeat       0.702812
alt-rock       0.754173
alternative    0.720030
ambient        0.237162
Name: energy, dtype: float64


In [19]:
#Top 10 most energetic genre
top10_energy_genre=average_energy_genre.sort_values(ascending=False).head(10)
print(top10_energy_genre)

track_genre
death-metal      0.931470
grindcore        0.924201
metalcore        0.914485
happy            0.910971
hardstyle        0.901246
drum-and-bass    0.876635
black-metal      0.874897
heavy-metal      0.874003
party            0.871237
j-idol           0.868677
Name: energy, dtype: float64


In [20]:
# 4.How many tracks is Bad Bunny on?

In [21]:
#How many tracks is Bad Bunny on?
df_csv_results.dropna(subset=['artists'], inplace=True)
Bad_Bunny_Tracks=df_csv_results[df_csv_results['artists'].str.contains('Bad Bunny')]
print("Number of tracks for Bad Bunny ", Bad_Bunny_Tracks.shape[0])

Number of tracks for Bad Bunny  416


In [22]:
# 5.Show the top 10 genres in terms of popularity sorted by their most popular track

In [23]:
# Group the DataFrame by genre and find the most popular track within each genre
top_genre_tracks=df_csv_results.groupby('track_genre').apply(lambda x: x.loc[x['popularity'].idxmax()])
print(top_genre_tracks.head(5))

             Unnamed: 0                track_id               artists  \
track_genre                                                             
acoustic              4  5vjLSffimiIP26QG5WcN2K      Chord Overstreet   
afrobeat           1100  1q8NdCAQ9QUjpYiqzdd3mv              Calle 13   
alt-rock           2003  2QjOHCTQ1Jl3zawyYOpxh6     The Neighbourhood   
alternative        3003  2QjOHCTQ1Jl3zawyYOpxh6     The Neighbourhood   
ambient            4002  0yc6Gst2xkRu0eMLeRMGCX  Cigarettes After Sex   

                              album_name       track_name  popularity  \
track_genre                                                             
acoustic                         Hold On          Hold On          82   
afrobeat     Calle 13 (Explicit Version)   Atrévete-Te-Te          75   
alt-rock                     I Love You.  Sweater Weather          93   
alternative                  I Love You.  Sweater Weather          93   
ambient             Cigarettes After Sex       Apo

In [24]:
# Sort the genres based on the popularity of their most popular track
top10_popular_genre=top_genre_tracks.sort_values(by='popularity', ascending=False)
print(top10_popular_genre.head(10))

             Unnamed: 0                track_id                  artists  \
track_genre                                                                
pop               81051  3nqQXoyQOWXiESFLlDF1hG     Sam Smith;Kim Petras   
dance             20001  3nqQXoyQOWXiESFLlDF1hG     Sam Smith;Kim Petras   
hip-hop           51664  2tTmW7RDtMQtBk7m2rYeSw         Bizarrap;Quevedo   
latino            68303  5ww2BF9slyYgNOk37BlC4u            Manuel Turizo   
reggaeton         89411  5ww2BF9slyYgNOk37BlC4u            Manuel Turizo   
edm               30003  4uUG5RXrOk84mYEfFvj3cK  David Guetta;Bebe Rexha   
latin             67356  5ww2BF9slyYgNOk37BlC4u            Manuel Turizo   
reggae            88410  5ww2BF9slyYgNOk37BlC4u            Manuel Turizo   
piano             79000  4h9wh7iOZ0GGn8QVp4RAOB              OneRepublic   
rock              91003  4h9wh7iOZ0GGn8QVp4RAOB              OneRepublic   

                                                    album_name  \
track_genre          

In [25]:
# Display the top 10 genres
top_10_genres = top10_popular_genre[['track_genre', 'track_name', 'popularity']].head(10)
print("Top 10 genres in terms of popularity sorted by their most popular track:")
print(top_10_genres)

Top 10 genres in terms of popularity sorted by their most popular track:
            track_genre                            track_name  popularity
track_genre                                                              
pop                 pop             Unholy (feat. Kim Petras)         100
dance             dance             Unholy (feat. Kim Petras)         100
hip-hop         hip-hop  Quevedo: Bzrp Music Sessions Vol. 52          99
latino           latino                            La Bachata          98
reggaeton     reggaeton                            La Bachata          98
edm                 edm                       I'm Good (Blue)          98
latin             latin                            La Bachata          98
reggae           reggae                            La Bachata          98
piano             piano                       I Ain't Worried          96
rock               rock                       I Ain't Worried          96


In [26]:
#Load data in Athena stagging to query the table

In [27]:
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
print(s3_staging_dir)

s3://sagemaker-us-east-1-652903355321/athena/staging


In [53]:
database_name = "Assignment2"
table_name = "MusicData"
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
import pandas as pd
pd.read_sql(statement, conn)
statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)

CREATE DATABASE IF NOT EXISTS Assignment2


,database_name
0,assgnaws
1,assignment2
2,default
3,dsoaws


In [54]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         srnbr int,
         track_id string,
         artists string,
         album_name string,
         track_name string,
         popularity int,
         duration_ms int,
         explicit string,
         danceability string,
         energy string,
         key int,
         loudness string,
         mode int,
         speechiness string,
         acousticness string,
         instrumentalness string,
         liveness string,
         valence string,
         tempo string,
         time_signature int,
         track_genre string
) ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
WITH SERDEPROPERTIES (
    'separatorChar' = ',',
    'quoteChar' = '"'
) LOCATION '{}' 
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name, s3_private_path_csv
)

print(statement)
pd.read_sql(statement, conn)

CREATE EXTERNAL TABLE IF NOT EXISTS Assignment2.MusicData(
         srnbr int,
         track_id string,
         artists string,
         album_name string,
         track_name string,
         popularity int,
         duration_ms int,
         explicit string,
         danceability string,
         energy string,
         key int,
         loudness string,
         mode int,
         speechiness string,
         acousticness string,
         instrumentalness string,
         liveness string,
         valence string,
         tempo string,
         time_signature int,
         track_genre string
) ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
WITH SERDEPROPERTIES (
    'separatorChar' = ',',
    'quoteChar' = '"'
) LOCATION 's3://sagemaker-us-east-1-652903355321/dataset/csv' 
TBLPROPERTIES ('skip.header.line.count'='1')


""


In [55]:
statement = """SELECT * FROM {}.{} LIMIT 100""".format(
    database_name, table_name
)

print(statement)
df = pd.read_sql(statement, conn)
df.head(5)

SELECT * FROM Assignment2.MusicData LIMIT 100


,srnbr,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,-6.746,0,0.143,0.0322,1.01e-06,0.358,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.42,0.166,...,-17.235,1,0.0763,0.924,5.56e-06,0.101,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.359,...,-9.734,1,0.0557,0.21,0.0,0.117,0.12,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.905,7.07e-05,0.132,0.143,181.74,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.443,...,-9.681,1,0.0526,0.469,0.0,0.0829,0.167,119.949,4,acoustic


In [34]:
# 1.List artist, track_name, and popularity for songs that have a popularity greater than or equal to 99

In [56]:
statement = """SELECT artists, track_name, popularity FROM {}.{} WHERE POPULARITY >= 99 LIMIT 100""".format(
    database_name, table_name
)
print(statement)
df = pd.read_sql(statement, conn)
df.head(5)

SELECT artists, track_name, popularity FROM Assignment2.MusicData WHERE POPULARITY >= 99 LIMIT 100


,artists,track_name,popularity
0,Sam Smith;Kim Petras,Unholy (feat. Kim Petras),100
1,Bizarrap;Quevedo,"Quevedo: Bzrp Music Sessions, Vol. 52",99
2,Sam Smith;Kim Petras,Unholy (feat. Kim Petras),100


In [76]:
#3.List artists with an average popularity of 92

In [57]:
statement = """SELECT artists, AVG(POPULARITY) as average_popularity FROM {}.{} GROUP BY artists HAVING AVG(popularity) = 92 """.format(
    database_name, table_name
)
print(statement)
df = pd.read_sql(statement, conn)
df.head(5)

SELECT artists, AVG(POPULARITY) as average_popularity FROM Assignment2.MusicData GROUP BY artists HAVING AVG(popularity) = 92 


,artists,average_popularity
0,Harry Styles,92.0
1,Rema;Selena Gomez,92.0


In [48]:
# 3.List the Top 10 most energetic genres

In [64]:
statement = """SELECT track_genre, AVG(CAST(energy AS DOUBLE)) as top_energetic_genre FROM {}.{} GROUP BY track_genre ORDER BY top_energetic_genre DESC LIMIT 10 """.format(
    database_name, table_name
)
print(statement)
df = pd.read_sql(statement, conn)
df.head(10)

SELECT track_genre, AVG(CAST(energy AS DOUBLE)) as top_energetic_genre FROM Assignment2.MusicData GROUP BY track_genre ORDER BY top_energetic_genre DESC LIMIT 10 


,track_genre,top_energetic_genre
0,death-metal,0.931470
1,grindcore,0.924201
2,metalcore,0.914485
3,happy,0.910971
4,hardstyle,0.901246
5,drum-and-bass,0.876635
6,black-metal,0.874897
7,heavy-metal,0.874003
8,party,0.871237
9,j-idol,0.868677


In [ ]:
# 4.How many tracks is Bad Bunny on?

In [74]:
statement = """SELECT count(artists) FROM {}.{} WHERE artists like '%Bad Bunny%' """.format(
    database_name, table_name
)
print(statement)
df = pd.read_sql(statement, conn)
df.head(10)
print("Number of tracks for Bad Bunny:\n ", df)

SELECT count(artists) FROM Assignment2.MusicData WHERE artists like '%Bad Bunny%' 
Number of tracks for Bad Bunny:
     _col0
0    416


In [75]:
#Show the top 10 genres in terms of popularity sorted by their most popular track

In [82]:
statement = """SELECT track_genre , MAX(popularity) FROM {}.{} GROUP BY track_genre ORDER BY MAX(popularity) DESC LIMIT 10 """.format(
    database_name, table_name
)
print(statement)
df = pd.read_sql(statement, conn)
df.head(10)

SELECT track_genre , MAX(popularity) FROM Assignment2.MusicData GROUP BY track_genre ORDER BY MAX(popularity) DESC LIMIT 10 


,track_genre,_col1
0,dance,100
1,pop,100
2,hip-hop,99
3,latin,98
4,reggae,98
5,reggaeton,98
6,edm,98
7,latino,98
8,rock,96
9,piano,96


In [83]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>